In [113]:
import pymongo
import pandas as pd

In [114]:
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [117]:
def toGeoJSON(lista):
    return {
            "type": "Point",
            "coordinates":[lista[1],lista[0]]
        }

def getValuatio(funding_round):
    valor = 0
    for lista in funding_round:
        valor = lista['raised_amount']
    return valor

def convertRestaurant(company):
    lista_dict = []
    for index in range(len(company['offices'])):
        for key in company['offices']:
            if key['latitude'] != None and key['longitude'] != None:
                comp_off = {
                "name": company["name"],
                "category_code": company["category_code"],
                "number_of_employees": company["number_of_employees"],
                "founded_year": company["founded_year"],
                "valuation": getValuatio(company['funding_rounds']),
                "position": toGeoJSON([key['latitude'],key['longitude']])   
                }
                lista_dict.append(comp_off)
        return lista_dict

cursor = db.companies.find({"$and": [ { "offices.latitude": {"$exists": True} },{ "offices.longitude": {"$exists": True} }, { "founded_year": {"$gt": 1995} }, {"deadpooled_year": None}  ] },{"_id":0})
ofiComp = list(map(lambda r: convertRestaurant(r),cursor))
geoCompanies = [element for lista in ofiComp for element in lista]


In [118]:
df = pd.DataFrame(geoCompanies)
df.head()

,category_code,founded_year,name,number_of_employees,position,valuation
0,news,2004,Digg,60.0,"{'type': 'Point', 'coordinates': [-122.394523,...",5000000.0
1,web,1999,Postini,NaN,"{'type': 'Point', 'coordinates': [-122.247573,...",0.0
2,web,2006,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,...",5000000.0
3,web,2003,Gizmoz,NaN,"{'type': 'Point', 'coordinates': [-122.169472,...",5300000.0
4,web,2002,StumbleUpon,NaN,"{'type': 'Point', 'coordinates': [-122.419204,...",NaN


In [119]:
df.to_json("comp_with_coords.json", orient="records", lines=True)

In [120]:
cursor = db.clean.find({
  "position": {
    "$near": {
      "$geometry": {
        "type": "Point",
        "coordinates": [-122.394523, 37.764726] 
      },
      "$minDistance": 0,
      "$maxDistance":10000000
    }
  }
})

df = pd.DataFrame(list(cursor))
df.head()

OperationFailure: error processing query: ns=companies.cleanTree: GEONEAR  field=position maxdist=1e+07 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error: unable to find index for $geoNear query

In [ ]:
cursor = db.clean.find({
  "position": {
    "$near": {
      "$geometry": {
        "type": "Point",
        "coordinates": [-74.1080159, 40.7046139]
      },
      "$minDistance": 0,
      "$maxDistance":10000
    }
  }
})

import pandas as pd
df = pd.DataFrame(list(cursor))
df.head()

In [ ]:
category_code
deadpooled_year
founded_year
funding_rounds     raised_amount
number_of_employees
offices      latitude
            longitude
    
    "offices": 
        deadpoolyear: None
            
            
def toGeoJSON(lista):
    for i in lista:
        return {
            "type":"Point",
            "coordinates":(i['longitude'],i['latitude'])
        }

def convertRestaurant(company):
    for diccio in company["offices"]:
            
        return {
        "name": company["name"],
        "homepage_url": company["homepage_url"],
        "twitter_username": company["twitter_username"],
        "category_code": company["category_code"],
        "number_of_employees": company["number_of_employees"],
        "founded_year": company["founded_year"],     
        "valuation": company["ipo"]['valuation_amount']     
        "position": toGeoJSON(company["offices"])
    }

cursor = db.companies.find({"name": "AddThis"}).limit(5)
geoCompanies = list(map(lambda r: convertRestaurant(r),cursor))
geoCompanies
{$and: [ { "offices.latitude": {"$exists": true} },{ "offices.longitude": {"$exists": true} } ] }
{$and: [ { "offices.latitude": {"$exists": true} },{ "offices.longitude": {"$exists": true} }, { founded_year: {"$gt": 1995} } ] }